# XGBoost

In [94]:
# https://www.kaggle.com/c/titanic

In [95]:
import sklearn
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score , roc_auc_score

In [96]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

In [97]:
# Загружаем данные из файлов
train = pd.read_csv('./titanic/train.csv')
test = pd.read_csv('./titanic/test.csv')

In [98]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [99]:
# Заполняем пропуски в данных медианными 
# значениями факторов на обучающей выборке
train_median = train.median(axis=1)
train_imp = train.fillna(train_median)
test_imp = test.fillna(train_median)

In [100]:
def age_class(row):
    if row['Age'] <= 10:
        return 'kind'
    elif ( row['Age'] > 10 ) and ( row['Age'] <= 17 ):
        return 'teenager'
    elif ( row['Age'] > 17 ) and ( row['Age'] <= 21 ):
        return 'youth'
    elif ( row['Age'] > 21 ) and ( row['Age'] <= 55 ):
        return 'mature'    
    elif ( row['Age'] > 55 ) and ( row['Age'] <= 75 ):
        return 'elderly'
    elif ( row['Age'] > 75 ):
        return 'old' 

In [101]:
train_imp['age_class'] = train_imp.apply( age_class, axis = 1 )
test_imp['age_class'] = test_imp.apply( age_class, axis = 1 )

In [102]:
# Бинаризуем категориальные признаки
CATEGORY_COL = ['Sex', 'Pclass', 'Embarked','age_class']
train_dummies = pd.get_dummies(train_imp, columns=CATEGORY_COL, drop_first=True)
test_dummies = pd.get_dummies(test_imp, columns=CATEGORY_COL, drop_first=True)

In [103]:
# Удаляем лишние столбцы
DROP_COL = ['PassengerId', 'Name', 'Ticket', 'Cabin']
TARGET_COL = 'Survived'
X_train = train_dummies.drop(DROP_COL + [TARGET_COL], axis=1)
y_train = train_dummies[TARGET_COL]
X_test = test_dummies.drop(DROP_COL, axis=1)

In [104]:
test = pd.read_csv('titanic/test.csv')

In [105]:
X_train.head()

,Age,SibSp,Parch,Fare,Sex_male,Pclass_2,Pclass_3,Embarked_Q,Embarked_S,age_class_kind,age_class_mature,age_class_old,age_class_teenager,age_class_youth
0,22.0,1,0,7.2500,1,0,1,0,1,0,1,0,0,0
1,38.0,1,0,71.2833,0,0,0,0,0,0,1,0,0,0
2,26.0,0,0,7.9250,0,0,1,0,1,0,1,0,0,0
3,35.0,1,0,53.1000,0,0,0,0,1,0,1,0,0,0
4,35.0,0,0,8.0500,1,0,1,0,1,0,1,0,0,0


In [106]:
for_drop = ["Pclass_3",
            "Pclass_2",
#            "Age", 
            "SibSp",
            "Parch",
            "Embarked_Q",
            "Embarked_S"#,
#            "FamilySize_1",
#            "FamilySize_4",
           ]
#for_drop = []
X_train.drop(for_drop, inplace=True, axis=1) #"clusters_0",  "Parch", "Miss","Ms", "Mrs"
X_test.drop(for_drop, inplace=True, axis=1)

In [107]:
X_train.head()

,Age,Fare,Sex_male,age_class_kind,age_class_mature,age_class_old,age_class_teenager,age_class_youth
0,22.0,7.2500,1,0,1,0,0,0
1,38.0,71.2833,0,0,1,0,0,0
2,26.0,7.9250,0,0,1,0,0,0
3,35.0,53.1000,0,0,1,0,0,0
4,35.0,8.0500,1,0,1,0,0,0


In [108]:
#for_drop = ["clusters_0", 
#            "clusters_1" ,
#            "IsAlone" ,
#            "Age_4.0", 
#            "FamilySize_7", 
#            "FamilySize_8",
#            "FamilySize_11",
#            "Pclass_2",
#            "FamilySize_2", 
#            "Title_3",
#            "FamilySize_3",
#            "Title_2",
#            "Title_4",
#            "FamilySize_1",
#            "FamilySize_4",]
#for_drop = []
#X_train.drop(for_drop, inplace=True, axis=1) #"clusters_0",  "Parch", "Miss","Ms", "Mrs"
#X_test.drop(for_drop, inplace=True, axis=1)

## Шаг 1: Зафиксируем learning_rate и параметры дерева и подберём n_estimators

Параметры:

* **max_depth**. Как указанов в таблице выше, обычно варьируется в интервале от 3 до 10 (но от задачи к задаче значения могут меняться). В качестве начального значения обычно используют 5
* **min_child_weight**. Если выборка сильно несбалансирована, то лучше выбрать значение "1". Иначе лучше выбрать значение "2" и зафиксировать
* **gamma**. Обычно выставляют значение в интервале от 0 до 0.2 и фиксируют. В дальнейшем этот параметр всегда можно затюнить отдельно
* **subsample, colsample_bytree**. Выставим 0.8 и зафиксируем. Можно также проварьировать в интервале 0.5-0.9.
* **scale_pos_weight**. Выставляется в зафисимости от соотношения классов в выборке и фиксируется

In [109]:
import xgboost

from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV

ones_ratio = y_train[y_train == 1].shape[0] * 1.0 / y_train[y_train == 0].shape[0] # посчитаем соотношение между классами

param_grid = {
    # параметры ансамбля
    'n_estimators': list(np.arange(10, 1000, 50)),#[10, 30, 50, 100, 200, 400, 600, 1000],
    'learning_rate': [0.01],
    
    # параметры дерева
    'max_depth': [5],
    'min_child_weight': [2],
    'gamma': [0.1],
    'subsample': [0.8],
    'colsample_bytree': [0.8],
    'scale_pos_weight': [ones_ratio],
    
    # параметры регуляризации
    'reg_alpha': [0.0],
    'reg_lambda': [1.0]
}

cv = KFold(n_splits=4, shuffle=True)

clf = xgboost.XGBClassifier()
gs = GridSearchCV(clf, param_grid, scoring='roc_auc', cv=cv, verbose=0)

gs.fit(X_train, y_train)
best_params = gs.best_estimator_.get_params()
print('Best score (AUC): ', gs.best_score_)
print('Best params: ')
best_params

Best score (AUC):  0.83897166601
Best params: 


{'base_score': 0.5,
 'colsample_bylevel': 1,
 'colsample_bytree': 0.8,
 'gamma': 0.1,
 'learning_rate': 0.01,
 'max_delta_step': 0,
 'max_depth': 5,
 'min_child_weight': 2,
 'missing': None,
 'n_estimators': 960,
 'nthread': -1,
 'objective': 'binary:logistic',
 'reg_alpha': 0.0,
 'reg_lambda': 1.0,
 'scale_pos_weight': 0.6229508196721312,
 'seed': 0,
 'silent': True,
 'subsample': 0.8}

## Шаг 2. Подбираем параметры дерева
* **max_depth** - будем варьировать от 3 до 10 с шагом 2
* **min_child_weight** - от 1 до 6 с шагом 2

In [110]:
param_grid = {
    'max_depth': range(3, 10, 2),
    'min_child_weight': range(1, 6, 2)
}

clf = xgboost.XGBClassifier(**best_params) # в качестве отправной точки возьмём модель с наилучшими параметрами предыдущего шага

gs = GridSearchCV(clf, param_grid, scoring='roc_auc', cv=cv, verbose=0)

gs.fit(X_train, y_train)
best_params = gs.best_estimator_.get_params()
print('Best score (AUC): ', gs.best_score_)
print('Best params: ')
best_params

Best score (AUC):  0.854343773162
Best params: 


{'base_score': 0.5,
 'colsample_bylevel': 1,
 'colsample_bytree': 0.8,
 'gamma': 0.1,
 'learning_rate': 0.01,
 'max_delta_step': 0,
 'max_depth': 7,
 'min_child_weight': 1,
 'missing': None,
 'n_estimators': 960,
 'nthread': -1,
 'objective': 'binary:logistic',
 'reg_alpha': 0.0,
 'reg_lambda': 1.0,
 'scale_pos_weight': 0.6229508196721312,
 'seed': 0,
 'silent': True,
 'subsample': 0.8}

## Шаг 3. Подбираем gamma (критерий создания поддерева)
* **gamma** - от 0 до 0.5 с шагом 0.1

In [111]:
param_grid = {
    'gamma': [0.1*i for i in range(12)]
}

clf = xgboost.XGBClassifier(**best_params)

gs = GridSearchCV(clf, param_grid, scoring='roc_auc', cv=cv, verbose=0)

gs.fit(X_train, y_train)
best_params = gs.best_estimator_.get_params()
print('Best score (AUC): ', gs.best_score_)
print('Best params: ')
best_params

Best score (AUC):  0.848785812599
Best params: 


{'base_score': 0.5,
 'colsample_bylevel': 1,
 'colsample_bytree': 0.8,
 'gamma': 0.7000000000000001,
 'learning_rate': 0.01,
 'max_delta_step': 0,
 'max_depth': 7,
 'min_child_weight': 1,
 'missing': None,
 'n_estimators': 960,
 'nthread': -1,
 'objective': 'binary:logistic',
 'reg_alpha': 0.0,
 'reg_lambda': 1.0,
 'scale_pos_weight': 0.6229508196721312,
 'seed': 0,
 'silent': True,
 'subsample': 0.8}

## Шаг 4. Затюним subsample и colsample_bytree
* **subsample** - от 0.5 до 1.0 с шагом 0.1
* **colsample_bytree** - от 0.5 до 1.0 с шагом 0.1

In [112]:
param_grid = {
    'subsample': [0.5 + 0.1*i for i in range(6)],
    'colsample_bytree': [0.5 + 0.1*i for i in range(6)]
}

clf = xgboost.XGBClassifier(**best_params)

gs = GridSearchCV(clf, param_grid, scoring='roc_auc', cv=cv, verbose=0)

gs.fit(X_train, y_train)
best_params = gs.best_estimator_.get_params()
print('Best score (AUC): ', gs.best_score_)
print('Best params: ')
best_params

Best score (AUC):  0.855820335761
Best params: 


{'base_score': 0.5,
 'colsample_bylevel': 1,
 'colsample_bytree': 0.5,
 'gamma': 0.7000000000000001,
 'learning_rate': 0.01,
 'max_delta_step': 0,
 'max_depth': 7,
 'min_child_weight': 1,
 'missing': None,
 'n_estimators': 960,
 'nthread': -1,
 'objective': 'binary:logistic',
 'reg_alpha': 0.0,
 'reg_lambda': 1.0,
 'scale_pos_weight': 0.6229508196721312,
 'seed': 0,
 'silent': True,
 'subsample': 1.0}

## Шаг 5. Регуляризация
* **reg_alpha** [1e-5, 1e-2, 0.1, 1, 100]
* **reg_lambda** [1e-5, 1e-2, 0.1, 1, 100]

In [113]:
param_grid = {
    'reg_alpha': [1e-5, 1e-2, 0.1, 0.2, 0.3, 0.5, 1, 2, 3],
    'reg_lambda': [1e-5, 1e-2, 0.1, 0.2, 0.3, 0.5, 1, 2, 3]
}

clf = xgboost.XGBClassifier(**best_params)

gs = GridSearchCV(clf, param_grid, scoring='roc_auc', cv=cv, verbose=0)

gs.fit(X_train, y_train)
best_params = gs.best_estimator_.get_params()
print('Best score (AUC): ', gs.best_score_)
print('Best params: ')
best_params

Best score (AUC):  0.848155171288
Best params: 


{'base_score': 0.5,
 'colsample_bylevel': 1,
 'colsample_bytree': 0.5,
 'gamma': 0.7000000000000001,
 'learning_rate': 0.01,
 'max_delta_step': 0,
 'max_depth': 7,
 'min_child_weight': 1,
 'missing': None,
 'n_estimators': 960,
 'nthread': -1,
 'objective': 'binary:logistic',
 'reg_alpha': 0.01,
 'reg_lambda': 3,
 'scale_pos_weight': 0.6229508196721312,
 'seed': 0,
 'silent': True,
 'subsample': 1.0}

## Шаг 6. Learning rate
Чем меньше у нас **n_estimators** в ансамбле, тем быстрее нам нужно двигаться с каждым шагом (добавлением нового классификатора), т.е. делать больший **learning_rate**. Обычно **learning rate** варьируют так, чтобы произведение **n_estimators** x **learning_rate** оставалось инвариантным

In [114]:
import numpy as np


clf = xgboost.XGBClassifier(**best_params)
best_n_estimators = clf.get_params()['n_estimators'] # возьмём наилучшие значения n_estimators с предыдущего шага
best_learning_rate = best_params['learning_rate'] # аналогичная запись
invariant_composition = best_n_estimators * best_learning_rate
n_estimators_range = np.arange(10, 1000, 50)#[10, 30, 100, 200, 400, 600, 800, 1000]

best_score = gs.best_score_ # возьмём наилучшее качество с предыдущего шага

for n_estimators in n_estimators_range:
    learning_rate = invariant_composition / n_estimators
    clf.set_params(n_estimators=n_estimators, learning_rate=learning_rate)
    aucs = []
    for train_idx, test_idx in cv.split(X_train):
        X_train_fold, X_test_fold = X_train.iloc[train_idx], X_train.iloc[test_idx]
        y_train_fold, y_test_fold = y_train.iloc[train_idx], y_train.iloc[test_idx]
        clf.fit(X_train_fold, y_train_fold, verbose=0)
        preds = clf.predict_proba(X_test_fold)
        auc = roc_auc_score(y_test_fold, preds[:, 1])
        aucs.append(auc)
    auc = np.mean(auc)
    if auc > best_score:
        best_n_estimators = n_estimators
        best_learning_rate = learning_rate
        best_score = auc
        
best_params['n_estimators'] = best_n_estimators
best_params['learning_rate'] = best_learning_rate

print('Best score (AUC): ', best_score)
print('Best params: ')
best_params

Best score (AUC):  0.893118466899
Best params: 


{'base_score': 0.5,
 'colsample_bylevel': 1,
 'colsample_bytree': 0.5,
 'gamma': 0.7000000000000001,
 'learning_rate': 0.087272727272727266,
 'max_delta_step': 0,
 'max_depth': 7,
 'min_child_weight': 1,
 'missing': None,
 'n_estimators': 110,
 'nthread': -1,
 'objective': 'binary:logistic',
 'reg_alpha': 0.01,
 'reg_lambda': 3,
 'scale_pos_weight': 0.6229508196721312,
 'seed': 0,
 'silent': True,
 'subsample': 1.0}

# Получили Best score (AUC):  0.935079225352

# Выведем важность признаков

In [115]:
import operator
z = sorted(zip(X_train.columns, clf.feature_importances_), key=operator.itemgetter(1), reverse=True)

for feature_name, feature_importance in z:
    print('Feature: "%s"\tFeature importance: %.4f' % (feature_name, feature_importance))

Feature: "Fare"	Feature importance: 0.5550
Feature: "Age"	Feature importance: 0.3191
Feature: "Sex_male"	Feature importance: 0.0556
Feature: "age_class_kind"	Feature importance: 0.0232
Feature: "age_class_mature"	Feature importance: 0.0201
Feature: "age_class_youth"	Feature importance: 0.0182
Feature: "age_class_teenager"	Feature importance: 0.0087
Feature: "age_class_old"	Feature importance: 0.0000


# Обучим модель с лучшими параметрами и отправим на kaggle

In [116]:
best_params = {'base_score': 0.5,
 'colsample_bylevel': 1,
 'colsample_bytree': 0.9,
 'gamma': 0.1,
 'learning_rate': 0.0076923076923076927,
 'max_delta_step': 0,
 'max_depth': 5,
 'min_child_weight': 1,
 'missing': None,
 'n_estimators': 260,
 'nthread': -1,
 'objective': 'binary:logistic',
 'reg_alpha': 0.2,
 'reg_lambda': 0.3,
 'scale_pos_weight': 1.0,
 'seed': 0,
 'silent': True,
 'subsample': 1.0}

clf = xgboost.XGBClassifier(**best_params)


In [117]:
clf.fit(X_train, y_train)

XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.9,
       gamma=0.1, learning_rate=0.007692307692307693, max_delta_step=0,
       max_depth=5, min_child_weight=1, missing=None, n_estimators=260,
       nthread=-1, objective='binary:logistic', reg_alpha=0.2,
       reg_lambda=0.3, scale_pos_weight=1.0, seed=0, silent=True,
       subsample=1.0)

In [118]:
predictions = clf.predict(X_test)
predictions

array([0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1,
       0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0,
       0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0,
       1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0,
       1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1,
       1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0,
       1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1,
       0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1,
       0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0,
       0, 0,

In [119]:
submussion = 'PassengerId,Survived\n'
submussion += "\n".join(["{},{}".format(pid, prediction) for pid, prediction in zip(test.PassengerId, predictions)])

In [120]:
with open('submission.txt', 'w') as file:
    file.write(submussion)